<a href="https://colab.research.google.com/github/youneslebboukh/text-summarization/blob/main/txt_summ_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"youneslbk","key":"ff30355568a25aef64e5c064a14e36fc"}'}

In [3]:
 !mkdir -p ~/.kaggle
 !cp kaggle.json ~/.kaggle/

In [4]:
  !chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

100% 6.18G/6.19G [01:30<00:00, 39.8MB/s]
100% 6.19G/6.19G [01:30<00:00, 73.6MB/s]


In [6]:
!pip install zip_files

In [10]:
from zipfile import ZipFile
file_name="CORD-19-research-challenge.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [11]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
metadata = pd.read_csv("/content/metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [13]:
import json

In [14]:
import glob

In [63]:
p="/content/document_parses"
all_json = glob.glob(f'{p}/**/*.json', recursive=True)
len(all_json)
#print(all_json[0])

240409

In [79]:
full_body_text=[]
for i in range(5) :
  with open(all_json[i]) as file :
    content = json.load(file)
    abstract = []
    body_text = []
    # Body text
    for entry in content['body_text']:
      body_text.append(entry['text'])
    full_body_text.append(body_text)
    body_text = '\n'.join(body_text)
   
      # Abstract
    #for entry in content['abstract']:
    # abstract.append(entry['text'])
    #abstract = '\n'.join(abstract)

In [92]:
print(body_text)
print("***************************************************************************************")
for i in full_body_text :
  print(i)

All viruses require host translational machinery in order to replicate. To make use of this machinery while avoiding detection, viruses have evolved a number of unconventional translational strategies (Gale et al, 2000), including internal ribosome entry sites (Jang et al, 1988), leaky scanning (Schwartz et al, 1990), ribosome shunting (Schmidt-Puchta et al, 1997), reinitiation (Hemmings-Mieszczak et al, 1997), and programmed ribosomal frameshifting (PRF) (Gesteland and Atkins, 1996). Many of these strategies are unique to viruses, and thus are potential antiviral targets (Harford, 1995).
During PRF, the translational machinery slips backward or forward by one nucleotide while decoding a so-called ‘slippery sequence,' a heptanucleotide sequence of the form XXXYYYZ in the mRNA transcript (Figure 1). Outside of PRF, frameshifting is a rare event, occurring fewer than 5 × 10−5 times per codon (Kurland, 1992). PRF can increase the occurrence of frameshifting by several orders of magnitude,

In [83]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
    text=re.sub("(\\t)", ' ', text) #remove escape charecters
    text=re.sub("(\\r)", ' ', text) 
    text=re.sub("(\\n)", ' ', text)
    text= re.sub("(\s+)",' ',text) #remove multiple space
    # return lowercase text
    return text.lower()

In [127]:
from nltk.tokenize import sent_tokenize
full_body_text2=[]

In [128]:
for i in full_body_text :
  txt = preprocessing(i[0])
  t=sent_tokenize(txt)
  full_body_text2.append(t)

In [130]:
for s in full_body_text2 :
  print(s)
len(full_body_text2) 

['emerging and re-emerging viral infections are becoming severe global public health problems in the current century.', 'in february 2003 an outbreak of severe acute respiratory syndrome sars caused by sars coronavirus sars-cov was reported in guangdong china ksiazek et al 2003 .', 'the sars-cov infected 8096 cases and 774 deaths worldwide.', 'in march 2009 a novel influenza virus h1n1pdm emerged in the united states and mexico.', 'h1n1pdm obtained the capacity to transmit in humans and quickly spread to more than 214 countries hendrickson and matthay 2013 .', 'thereafter the h1n1 infection became a seasonal virus circulating over the world dawood et al 2009 .', 'middle east respiratory syndrome mers coronavirus was isolated from patients who developed acute pneumonia and renal failure in 2012 from saudi arabia zaki et al 2012 .', 'one big secondary outbreak with 186 confirmed cases was in south korea in 2015. up to january 2020 more than 2500 confirmed cases were reported with a case 

5